In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert
NbViol = 6
NbMems = 256
NumViolon = np.load('./../results/NumViolon.npz')['NumViolon']



## Analyse de la RI Marteau->Mems comme RIR
Les indices standards extraits de l'analyse de la réponse impulsionnelle d'une salle incluent généralement les suivants :

1. **Temps de Réverbération (RT60)** : Le temps nécessaire pour que le niveau sonore diminue de 60 dB après l'arrêt de la source sonore.

2. **Early Decay Time (EDT)** : Le temps nécessaire pour que le niveau sonore diminue de 10 dB, extrapolé pour estimer le RT60.

3. **Clarity (C50, C80)** : Mesure de la clarté du son, définie comme le rapport entre l'énergie sonore précoce (dans les 50 ms ou 80 ms) et l'énergie sonore tardive.

4. **Définition (D50)** : Le pourcentage de l'énergie sonore totale qui arrive dans les 50 ms suivant l'arrivée du son direct.

5. **Centre Temporel (Ts)** : Le moment où l'énergie cumulative atteint 50 % de l'énergie totale.

6. **Rapport Signal/Bruit (SNR)** : Le rapport entre le niveau du signal utile et le niveau du bruit de fond.


In [3]:
RIhm = np.load('./../results/ViolinsFRFsAndRIs.npz')['RIhm']
tW = np.load('./../results/ViolinsFRFsAndRIs.npz')['tW']
tWMax = 0.015
nm = 10
Dyn = 40

RIhmN = RIhm / np.max(np.abs(RIhm), axis=1)[:,None,:]
Env = np.abs(hilbert(RIhmN, axis=1))
EnvdB = 20*np.log10(Env)
fig41, ax41 = plt.subplots(6,1,figsize=(10,10))
imaxRI = np.argmax(Env, axis=1)
for nf in range(NbViol):
    #RI    
#    ax41[nf].plot(1000*(tW-tW[imaxRI[nf,nm]]), EnvdB[nf, :, nm], 'k', lw = 2)    
    ax41[nf].plot(1000*tW, EnvdB[nf, :, nm], 'k', lw = 2)    

    ax41[nf].set_xlim([0, 50])
    ax41[nf].set_ylim([-Dyn, 0])

    ax41[nf].set_ylabel('dB')
    ax41[nf].text(1000*tWMax, 0.5, f'Violin #{NumViolon[nf]:d}', fontsize=12, ha='right')

    ax41[nf].spines['top'].set_visible(False)
    ax41[nf].spines['right'].set_visible(False)
    ax41[nf].xaxis.set_ticks_position('bottom')
    ax41[nf].yaxis.set_ticks_position('left')
    # Remove tick labels
    if nf < 5:
        ax41[nf].set_xticklabels([])
    else:    
        ax41[nf].set_xlabel('Time (ms)')
    if not nf : 
        ax41[nf].set_title('Impulse Response Enveloppe Mems#10/Ham')
fig41.tight_layout()


FileNotFoundError: [Errno 2] No such file or directory: './../results/ViolinsFRFsAndRIs.npz'

In [ ]:
LastOver10 = np.zeros((NbViol,NbMems))
LastOver20 = np.zeros((NbViol,NbMems))
LastOver30 = np.zeros((NbViol,NbMems))
for v in range(NbViol):
    for m in range(NbMems):
        for i in range(EnvdB.shape[1]):
            if EnvdB[v,i,m] > -10 :
                LastOver10[v,m] = tW[i]*1000
            if EnvdB[v,i,m] > -20 :
                LastOver20[v,m] = tW[i]*1000
            if EnvdB[v,i,m] > -30 :
                LastOver30[v,m] = tW[i]*1000

means10 = np.mean(LastOver10, axis=1)
std_devs10 = np.std(LastOver10, axis=1)
means20 = np.mean(LastOver20, axis=1)
std_devs20 = np.std(LastOver20, axis=1)
means30 = np.mean(LastOver30, axis=1)
std_devs30 = np.std(LastOver30, axis=1)

# X-axis positions for the violins
x_positions = np.arange(1, NbViol + 1)

                
# Scatter plot with error bars
plt.figure(figsize=(8, 5))
plt.errorbar(x_positions, means10, yerr=std_devs10, fmt='o', capsize=5, label='T10', color='blue')
plt.errorbar(x_positions, means20, yerr=std_devs20, fmt='o', capsize=5, label='T20', color='red')
plt.errorbar(x_positions, means30, yerr=std_devs30, fmt='o', capsize=5, label='T30', color='green')
# Add labels and title
plt.xticks(x_positions, [f"Violin {NumViolon[i-1]}" for i in x_positions])
plt.xlabel("Violins")
plt.ylabel("Time (ms)")
plt.title("Decay Time")
plt.legend()
plt.grid()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()